In [0]:
CREATE OR REPLACE VIEW internet_sales_metric_view
  WITH METRICS
  LANGUAGE YAML
  COMMENT 'A Metric View for regional sales metrics.'
  AS $$
version: 0.1
source: adventureworks.adventureworks_dw.fact_internetsales

joins:
  - name: customer
    source: adventureworks.adventureworks_dw.dim_customer
    on: Ship_CustomerKey = customer.CustomerKey
  - name: date
    source: adventureworks.adventureworks_dw.dim_date
    on: Ship_DateKey = date.DateKey
  - name: product
    source: adventureworks.adventureworks_dw.dim_product
    on: source.ProductKey = product.ProductKey

dimensions:
  # internetsales dimensions
  - name: Sales Order Number
    expr: source.SalesOrderNumber
  - name: Sales Order ID
    expr: source.SalesOrderID
  - name: Sales Order Detail ID
    expr: source.SalesOrderDetailID
  - name: Order Qty
    expr: source.OrderQty
  - name: Total Due
    expr: source.TotalDue
  
  # Customer dimensions
  - name: Customer ID
    expr: customer.CustomerID
  - name: AddressID
    expr: customer.AddressID
  - name: Customer Name
    expr: customer.FirstName || ' ' || customer.LastName
  - name: City
    expr: customer.City
  - name: State
    expr: customer.StateProvince
  - name: Zip Code
    expr: customer.PostalCode
  - name: Country
    expr: customer.CountryRegion
  - name: Email
    expr: customer.EmailAddress
  - name: Phone
    expr: customer.Phone

  # Date dimensions
  - name: Date
    expr: date.Date
  - name: Day of Week
    expr: date.DayofWeek
  - name: Year
    expr: date.Year
  - name: Month
    expr: date.YearMonthInt

  # Product dimensions
  - name: Product ID
    expr: product.ProductID
  - name: Product Alt Key
    expr: product.ProductAlternateKey
  - name: Product Name
    expr: product.EnglishProductName
  - name: Product Description
    expr: product.EnglishDescription
  - name: Product Color
    expr: product.Color
  - name: Product Class
    expr: product.Class
  - name: Product Category ID
    expr: product.ProductCategoryID
  - name: Product Category
    expr: product.ProductCategory

measures:
  - name: Total Sales
    expr: SUM(TotalDue)
  - name: Order Quantity
    expr: SUM(OrderQty)
  - name: Tax
    expr: SUM(TaxAmt)
  - name: Order Count
    expr: COUNT(SalesOrderID)
  - name: Average Line Total
    expr: AVG(LineTotal)
  - name: Distinct Customers
    expr: COUNT(DISTINCT Ship_CustomerKey)
  - name: Total Sales t30d
    expr: SUM(TotalDue)
    window:
      - order: Date
        range: trailing 30 day
        semiadditive: last
 $$;

In [0]:
SELECT 
    `Product Category`,
    `Customer Name`,
    measure(`Total Sales`) AS Total_Sales
  FROM adventureworks.adventureworks_dw.internet_sales_metric_view
  WHERE Month = 202507
  GROUP BY ALL